In [ ]:
import mne, fooof, numpy as np, pandas as pd, os, glob, sys
from IPython.display import display, Markdown
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [ ]:
projpath="/home/tmorshed/projects/def-johngrif/tmorshed/"

In [ ]:
datapath=os.path.join(projpath, 'data', 'nmod')
gitpath=os.path.join(projpath, 'bme1500')
if os.path.exists(datapath) and os.path.exists(gitpath):
    display(Markdown("success"))
else:
    display(Markdown('fix it'))

In [ ]:
os.chdir(datapath)

In [ ]:
def md(input):
    display(Markdown(input))
    
## DATA LOARDERS:

def finder():
    filesDepth5 = glob.glob('*/*/*/*/*')
    dirsDepth5 = filter(lambda f: os.path.isdir(f), filesDepth5)

    markers=np.sort(np.array([os.path.join(datapath, f) for f in filesDepth5 if f.endswith('.pkl')]))
    npzs=np.sort(np.array([os.path.join(datapath, f) for f in filesDepth5 if f.endswith('.npz')]))
    return markers, npzs

def load_to_df(markers, npzs, sno):
    _marker=pd.read_pickle(markers[sno])
    fs=_marker['fs']
    dt=1/fs


    with open(npzs[sno], 'rb') as f:
        _npz=np.load(f)
        _data=_npz.f.arr_0
        f.close()
    del _npz
    _df_data=pd.DataFrame(_data,
                          index=_marker['chNames'])
    _df_data.columns=_df_data.columns*dt

    a=pd.DataFrame(np.zeros((np.shape(_data)[-1]), dtype=str))
    a.index=a.index*dt

    _dfidx = pd.DataFrame([i['type'] for i in _marker['mrk']], index=[i['idx']*dt for i in _marker['mrk']])
    _dfidx=_dfidx.drop(labels=_dfidx.index[0])

    for i in _dfidx.index:
        a.loc[i]=_dfidx.loc[i]

    intdict={"":0,
            "s":1,
            "x":2,
            "na":3}

    for x in intdict:
        a=a.replace(x, int(intdict[x]))
    a=a.T
    a.index=['stim']

    _df_data=pd.concat([_df_data, a])
    _df_data.columns.names=['t']
    _df_data.index.names=['ch']
    return _marker, _df_data

def df_to_mne(_marker, _df_data):
    info = mne.create_info(ch_names=_marker['chNames'], sfreq=_marker['fs'], ch_types=(['eeg']*64)+(['misc']*7))
    info['subject_info']={"id":_marker['patientId']}
    raw=mne.io.RawArray(_df_data.iloc[:-1, :],info)
    raw.set_montage('standard_1020')
    return raw

In [ ]:
markers, npzs = finder()

In [ ]:
_marker, _df_data=load_to_df(markers, npzs, 0)

Plotting stimulations:

In [ ]:
b=np.array(_df_data.iloc[-1,:])
fig,ax=plt.subplots(figsize=(20,4))
plt.scatter(x=_df_data.columns,y=b, color="k", marker="s")
plt.xlabel('time (s)')
plt.title('stimulations offered')
plt.yticks(np.unique(b))
plt.ylabel('stimulation type')
display(pd.DataFrame(intdict, columns=intdict.keys(), index=intdict.values()).iloc[0, :])
plt.show()

In [ ]:
_marker.keys()

In [ ]:
_df_data.head()

In [ ]:
raw = df_to_mne(_marker, _df_data)

In [ ]:
_df_data.tail()